In [1]:
import pandas as pd
import pulp
import plotly.express as px



df_s=pd.read_excel("/workspaces/Accident-Opt-Strabag/DataFiles/sources_hosp_toll.xlsx")
df_dp=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/KMeans_clusters.csv")
df_s_old=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/old-ambulance_allotment.csv")
df_iterations_result=pd.DataFrame({'lat':df_s['Source_Latitude'],'lon':df_s['Source_Longitude']})
time_matrix_orsm=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/Distance_Time_Matrix/osrm_time_matrix.csv") #for osrm based
time_matrix_chainage=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/Distance_Time_Matrix/chainage_time_matrix.csv") #for chainage based
new_column_names = list(range(0, 16))
time_matrix_chainage.columns = new_column_names
time_matrix_orsm.columns = new_column_names

time_np_chainage=time_matrix_chainage.to_numpy()
time_np_orsm=time_matrix_orsm.to_numpy()

clusters=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/KMeans_clusters.csv")
weights=clusters['Accident_Frequency'].to_list()

number_of_iterations=6


P_values=[6,8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
r1=[20,18,16,12,10,8]
r2= [30,25,20,16,12,10] 
alpha=[0.8, 0.9, 1.0]
alpha_values = [0.8, 0.9, 1.0]
time=[time_np_chainage,time_np_orsm]
count=1

for l in range(number_of_iterations):

    for p in P_values:

        for t in range(len(time)):

            for a in alpha_values:

                time_np= time[t]
                if t==0:
                    t_matrix="chainage_time"
                else:
                    t_matrix="orsm_time"

                n=26 #number of demand points
                m=16 #potential sites ambulance location
                P=p #number of amnbulances to be located
                alpha=a #reliability Taking 80% coverage within r1 time
                a_i = weights #weights at each site
                t_ij= time_np #shortedt time to travel from i to j  TIME MATRIX
                r_1= r1[l] #primary coverage time (mins)
                r_2= r2[l] #secondary coverage time (mins) ASSUMED 
                p_j=[] #maximum number of ambulances at each site IT WILL BE 1 for our case now
                for i in range(m):
                    p_j.append(2)

                
                print(P,r_1,r_2,a)
                print("--------------------")


                demand_points=[]
                for i in range(0,26):
                    demand_points.append(i)
                source_locations=[] #the potential locations of the ambulances 
                for i in range(0,16):
                    source_locations.append(i)

                # Define the new binary variables z_ij to represent a_ij2 * x_j
                z = pulp.LpVariable.dicts("z", [(i, j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)
                #new variable to represent a_ij1*n_j 
                x=pulp.LpVariable.dicts("x",[(i,j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)


                z_i1 = pulp.LpVariable.dicts("z_i1", [i for i in demand_points], 0, 1, pulp.LpBinary)
                z_i2 = pulp.LpVariable.dicts("z_i2", [i for i in demand_points], 0, 1, pulp.LpBinary)
                a_ij1 = pulp.LpVariable.dicts("a_ij1", [(i, j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)
                a_ij2 = pulp.LpVariable.dicts("a_ij2", [(i, j) for i in demand_points for j in source_locations], 0, 1, pulp.LpBinary)
                n_j = pulp.LpVariable.dicts("n_j", source_locations, 0, 2, pulp.LpInteger)

                problem=pulp.LpProblem("Ambulance-Coverage-Loc",sense=pulp.LpMaximize)
                problem+=pulp.lpSum(a_i[i]*z_i2[i] for i in demand_points)

                #CONSTRAINTS

                for i in demand_points:
                    k_times = pulp.lpSum(1 for j in source_locations if t_ij[(i,j)] <= r_1 and n_j[j] != 0)
                    problem += k_times >= 2
                    problem += z_i2[i] == 1 if k_times >= 2 else z_i2[i] == 0

                for k in demand_points:
                    k_times = pulp.lpSum(1 for j in source_locations if t_ij[(k,j)] <= r_1 and n_j[j] != 0)
                    problem += k_times >= 1
                    problem += z_i1[k] == 1 if k_times >= 1 else z_i1[k] == 0


                for i in demand_points:
                    for j in source_locations:
                        if t_ij[(i,j)]<=r_2 and n_j[j]!=0:
                            problem+= a_ij2[(i,j)]==1
                            problem+= z[(i,j)]==n_j[j]
                        else:
                            problem+= a_ij2[(i,j)]==0
                            problem+=z[(i,j)]==0
                        if t_ij[(i,j)]<=r_1 and n_j[j]!=0:
                            problem+=a_ij1[(i,j)]==1
                            problem+=x[(i,j)]==n_j[j]
                        else:
                            problem+=a_ij1[(i,j)]==0
                            problem+=x[(i,j)]==0

                #x for a_ij1*n_j z for a_ij2*n_j

                # Modify the original constraint to use the new binary variables
                for i in demand_points:
                    #problem += pulp.lpSum(a_ij2[(i, j)]*n_j for j in source_locations if t_ij[(i, j)] <= r_2) >= 1 #-------------------------------->2
                    problem += pulp.lpSum(z[(i, j)] for j in source_locations) >= 1


                # Constraints
                for i in demand_points:
                    problem += pulp.lpSum(a_i[i]*z_i1[i] for j in source_locations if t_ij[(i,j)]<=r_1)>=  alpha* pulp.lpSum(a_i[i]) #---------------------------->3
                    #problem += pulp.lpSum(x[(i,j)] for j in source_locations  if t_ij[(i, j)] <= r_1) >= z_i1[i] + z_i2[i] #------------------------>4
                    problem += pulp.lpSum(x[(i,j)] for j in source_locations) >= z_i1[i] + z_i2[i]
                    #problem += pulp.LpConstraint(z_i2[i]<=z_i1[i]) #------------------------------------>5

                for i in demand_points:
                    problem += z_i2[i]<=z_i1[i]

                for j in range(m):
                    #problem += pulp.LpConstraint(n_j[j] <= p_j[j]) #-------------------------->7
                    problem += n_j[j] <= p_j[j]

                for j in range(m):
                    #problem+=pulp.LpConstraint(n_j[j]>=0) #---------------------------------->8
                    problem+=n_j[j]>=0


                problem += pulp.lpSum(n_j[j] for j in source_locations ) == P #------------------------------>6

                problem.solve()

                # Solve the problem
                status = problem.solve()


                # Save solution outputs to a file
                with open(f"/workspaces/Accident-Opt-Strabag/DataFiles/Optimised_Allotments/opt_iteration_{l+1}_P_{P}_alpha_{a}_time_{t_matrix}.txt", "w") as file:
                    file.write("Solution status: {}\n\n".format(pulp.LpStatus[status]))
                    for j in source_locations:
                        file.write(f"Ambulance at location {j}: {n_j[j].varValue} \n")
                    
                    file.write("\n")

                    for k in demand_points:
                        file.write(f"Demand point {k} primary coverage (z_i1): {z_i1[k].varValue}\n")
                        file.write(f"Demand point {k} secondary coverage (z_i2): {z_i2[k].varValue}\n")

                    file.write("\n")
                    file.write(f"Objective function value: {pulp.value(problem.objective)}\n \n")
                    file.write(f"P: {p} |\t")
                    file.write(f"r_1: {r_1} |\t")
                    file.write(f"r_2: {r_2} |\t")
                    file.write(f"alpha: {alpha} |\t")


        
                allotment_list=[]
                for n in source_locations:
                    allotment_list.append(int(n_j[n].varValue))

                df=pd.DataFrame({'optimised-ambulances':allotment_list,'lat':df_s['Source_Latitude'],'long':df_s['Source_Longitude']})
                df.to_csv(f"/workspaces/Accident-Opt-Strabag/DataFiles/Optimised_Allotments/opt_Iteration_{l+1}_P_{P}_alpha_{a}_time_{t_matrix}.csv") 


                #SAVE PLOTS FOR EACH ITERATION

                def plot_centroids_ambulance(data,df_s_old,df_s_opt ,title):
                    df_accidents = pd.DataFrame(data, columns=['Accident_LATITUDE', 'Accident_LONGITUDE', 'Accident_Frequency','Cluster'])
                    
                    fig = px.scatter_mapbox(df_accidents, lat="Accident_LATITUDE", lon="Accident_LONGITUDE", color="Accident_Frequency", size='Accident_Frequency',
                                            color_continuous_scale='Reds', size_max=25, zoom=8.5,
                                            mapbox_style="carto-positron", title=title, hover_name='Cluster')
                                            #,labels={'Accident_Frequency': 'Sum of Accident Frequency'})

                        # Add trace for trauma centers
                    trace_old = px.scatter_mapbox(df_s_old.loc[df_s_old['ambulances']==1], lat="Source_Latitude", lon="Source_Longitude",
                                                    color_discrete_sequence=['black'], 
                                                    size_max=15, zoom=8.5,
                                                    mapbox_style="carto-positron").data[0]
                    trace_old.name = 'Trauma Centers'
                    fig.add_trace(trace_old)


                        # Add trace for ambulances
                    trace_opt = px.scatter_mapbox(df_s_opt.loc[df_s_opt['optimised-ambulances']==1], lat="lat", lon="long",
                                                        color_discrete_sequence=['blue'], 
                                                        size_max=30, zoom=8.5,
                                                        mapbox_style="carto-positron").data[0]
                    trace_opt.name = 'Ambulances'
                    fig.add_trace(trace_opt)
                
                    fig.update_layout(legend=dict(
                    title=title,
                    orientation='h',  # horizontal legend
                    yanchor='bottom',
                    y=1.02,
                    xanchor='right',
                    x=1
                    ))

                    fig.show()



                df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list
                
                df_iterations_result.to_csv("/workspaces/Accident-Opt-Strabag/DataFiles/iteration_result/iterations_result.csv")
   
               
                print("Loop Count:",count)
                print("------------------")
                count+=1  

6 20 30 0.8
--------------------
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-efkon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/795fe27da4c84369ac971c21510524c5-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/795fe27da4c84369ac971c21510524c5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1910 COLUMNS
At line 8618 RHS
At line 10524 BOUNDS
At line 12257 ENDATA
Problem MODEL has 1905 rows, 1732 columns and 3217 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 107 - 0.00 seconds
Cgl0002I 682 variables fixed
Cgl0004I processed model has 47 rows, 16 columns (16 integer (16 of which binary)) and 447 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -107
Cbc0038I Before mini branch and bound, 16 integers at bound f

/tmp/ipykernel_68257/1763288750.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list
/tmp/ipykernel_68257/1763288750.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-efkon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/51b37ae551074a95a8b50c6d21b12e3c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/51b37ae551074a95a8b50c6d21b12e3c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1910 COLUMNS
At line 8546 RHS
At line 10452 BOUNDS
At line 12185 ENDATA
Problem MODEL has 1905 rows, 1732 columns and 3145 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 107 - 0.00 seconds
Cgl0002I 826 variables fixed
Cgl0004I processed model has 43 rows, 16 columns (16 integer (16 of which binary)) and 338 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -107
Cbc0038I Before mini branch and bound, 16 integers at bound fixed and 0 continuous
Cbc0038I Mi

/tmp/ipykernel_68257/1763288750.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list
/tmp/ipykernel_68257/1763288750.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-efkon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2278fe44bbb54993b1ca6b9647e22287-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/2278fe44bbb54993b1ca6b9647e22287-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1910 COLUMNS
At line 8546 RHS
At line 10452 BOUNDS
At line 12185 ENDATA
Problem MODEL has 1905 rows, 1732 columns and 3145 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 107 - 0.00 seconds
Cgl0002I 826 variables fixed
Cgl0004I processed model has 40 rows, 16 columns (16 integer (16 of which binary)) and 328 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -107
Cbc0038I Before mini branch and bound, 16 integers at bound fixed and 0 continuous
Cbc0038I Mi

...

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-efkon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/9d995ad7ba1045d482a620e2e56c164a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/9d995ad7ba1045d482a620e2e56c164a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1910 COLUMNS
At line 8250 RHS
At line 10156 BOUNDS
At line 11889 ENDATA
Problem MODEL has 1905 rows, 1732 columns and 2849 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-efkon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /t

/tmp/ipykernel_68257/1763288750.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list
/tmp/ipykernel_68257/1763288750.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-efkon/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c00371e41bca4edf9660696ddbda3da0-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/c00371e41bca4edf9660696ddbda3da0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1910 COLUMNS
At line 8250 RHS
At line 10156 BOUNDS
At line 11889 ENDATA
Problem MODEL has 1905 rows, 1732 columns and 2849 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Loop Count: 503
------------------
20 8 10 1.0
--------------------
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /workspaces/Accident-Opt-Strabag/venv-e

/tmp/ipykernel_68257/1763288750.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_iterations_result[f'i-{l+1} | P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}']=allotment_list


Loop Count: 504
------------------


In [4]:
df_iterations_result

,i-1 | P-6 | r1-20 | r2-30 | alpha-0.8 | Time Matrix-chainage_time,i-1 | P-6 | r1-20 | r2-30 | alpha-0.9 | Time Matrix-chainage_time,i-1 | P-6 | r1-20 | r2-30 | alpha-1.0 | Time Matrix-chainage_time,i-1 | P-6 | r1-20 | r2-30 | alpha-0.8 | Time Matrix-orsm_time,i-1 | P-6 | r1-20 | r2-30 | alpha-0.9 | Time Matrix-orsm_time,i-1 | P-6 | r1-20 | r2-30 | alpha-1.0 | Time Matrix-orsm_time,i-1 | P-8 | r1-20 | r2-30 | alpha-0.8 | Time Matrix-chainage_time,i-1 | P-8 | r1-20 | r2-30 | alpha-0.9 | Time Matrix-chainage_time,i-1 | P-8 | r1-20 | r2-30 | alpha-1.0 | Time Matrix-chainage_time,i-1 | P-8 | r1-20 | r2-30 | alpha-0.8 | Time Matrix-orsm_time,...,i-6 | P-19 | r1-8 | r2-10 | alpha-1.0 | Time Matrix-chainage_time,i-6 | P-19 | r1-8 | r2-10 | alpha-0.8 | Time Matrix-orsm_time,i-6 | P-19 | r1-8 | r2-10 | alpha-0.9 | Time Matrix-orsm_time,i-6 | P-19 | r1-8 | r2-10 | alpha-1.0 | Time Matrix-orsm_time,i-6 | P-20 | r1-8 | r2-10 | alpha-0.8 | Time Matrix-chainage_time,i-6 | P-20 | r1-8 | r2-10 | alpha-0.9 | Time Matrix-chainage_time,i-6 | P-20 | r1-8 | r2-10 | alpha-1.0 | Time Matrix-chainage_time,i-6 | P-20 | r1-8 | r2-10 | alpha-0.8 | Time Matrix-orsm_time,i-6 | P-20 | r1-8 | r2-10 | alpha-0.9 | Time Matrix-orsm_time,i-6 | P-20 | r1-8 | r2-10 | alpha-1.0 | Time Matrix-orsm_time
0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,2,1,1,1,2,2,2
1,0,0,0,1,1,1,1,1,1,0,...,1,2,2,2,1,1,1,2,2,2
2,1,1,1,0,0,0,1,1,1,0,...,1,2,2,2,1,1,1,2,2,2
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,0,0,0
4,1,1,1,0,0,0,0,0,0,1,...,1,0,0,0,1,1,1,0,0,0
5,0,0,0,0,0,0,1,1,1,1,...,1,0,0,0,1,1,1,0,0,0
6,0,0,0,1,1,1,0,0,0,0,...,1,0,0,0,1,1,1,0,0,0
7,0,0,0,1,1,1,1,1,1,1,...,1,2,2,2,1,1,1,2,2,2
8,1,1,1,0,0,0,1,1,1,0,...,1,2,2,2,1,1,1,2,2,2
9,0,0,0,0,0,0,1,1,1,0,...,1,0,0,0,1,1,1,0,0,0


In [ ]:
 plot_centroids_ambulance(df_dp,df_s_old,df, f'Fatal Accident Cluster and Ambulances Old- Black and Optimised-Blue P-{P} | r1-{r_1} | r2-{r_2} | alpha-{alpha} | Time Matrix-{t_matrix}')
